In [5]:
import cv2

def drawBoundingBox(img, bbox):
    x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2, 1)
    cv2.putText(img, "Track", (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 255, 0), 2)

cap = cv2.VideoCapture(0)
tracker = cv2.legacy.TrackerMOSSE_create()
success, img = cap.read()

bbox = cv2.selectROI("Select Region of Interest", img, False)
tracker.init(img, bbox)

while True:
    timer = cv2.getTickCount()
    success, img = cap.read()
    success, bbox = tracker.update(img)

    if success:
        drawBoundingBox(img, bbox)
    else:
        cv2.putText(img, "Lost", (5, 25), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 0, 0), 2)

    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(img, str(int(fps)), (5, 50), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 0, 0), 2)
    cv2.imshow("Track using different trackers", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


[ WARN:0@1237.041] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1237.041] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
